# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 21 2022 1:14PM,253608,19,8931616,"Emersa Waterbox, LLC",Released
1,Dec 21 2022 1:14PM,253608,19,8931617,"Emersa Waterbox, LLC",Released
2,Dec 21 2022 1:14PM,253608,19,8931618,"Emersa Waterbox, LLC",Released
3,Dec 21 2022 1:07PM,253607,22,636742,"NBTY Global, Inc.",Released
4,Dec 21 2022 12:53PM,253606,10,Yusen-8931602,Yusen – 3D Matrix,Released
5,Dec 21 2022 12:53PM,253606,10,Yusen-8931603,Yusen – 3D Matrix,Released
6,Dec 21 2022 12:52PM,253599,10,PRF-45233,Bio-PRF,Released
7,Dec 21 2022 12:52PM,253599,10,PRF-45240,Bio-PRF,Released
8,Dec 21 2022 12:52PM,253599,10,PRF-45245,Bio-PRF,Released
9,Dec 21 2022 12:52PM,253599,10,PRF-45248,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,253603,Released,1
32,253604,Released,1
33,253606,Released,2
34,253607,Released,1
35,253608,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253603,NaN,NaN,1.0
253604,NaN,NaN,1.0
253606,NaN,NaN,2.0
253607,NaN,NaN,1.0
253608,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253547,0.0,5.0,2.0
253560,1.0,10.0,77.0
253563,0.0,0.0,9.0
253564,0.0,0.0,1.0
253570,0.0,0.0,23.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253547,0,5,2
253560,1,10,77
253563,0,0,9
253564,0,0,1
253570,0,0,23


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253547,0,5,2
1,253560,1,10,77
2,253563,0,0,9
3,253564,0,0,1
4,253570,0,0,23


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253547,,5,2
1,253560,1,10,77
2,253563,,,9
3,253564,,,1
4,253570,,,23


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 21 2022 1:14PM,253608,19,"Emersa Waterbox, LLC"
3,Dec 21 2022 1:07PM,253607,22,"NBTY Global, Inc."
4,Dec 21 2022 12:53PM,253606,10,Yusen – 3D Matrix
6,Dec 21 2022 12:52PM,253599,10,Bio-PRF
18,Dec 21 2022 12:33PM,253604,10,"Nextsource Biotechnology, LLC"
19,Dec 21 2022 12:25PM,253603,21,"NBTY Global, Inc."
20,Dec 21 2022 12:19PM,253602,10,"Goodwin Biotechnology, Inc."
21,Dec 21 2022 12:10PM,253601,10,"Snap Medical Industries, LLC"
23,Dec 21 2022 12:03PM,253600,10,Bio-PRF
24,Dec 21 2022 12:01PM,253597,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 21 2022 1:14PM,253608,19,"Emersa Waterbox, LLC",,,3
1,Dec 21 2022 1:07PM,253607,22,"NBTY Global, Inc.",,,1
2,Dec 21 2022 12:53PM,253606,10,Yusen – 3D Matrix,,,2
3,Dec 21 2022 12:52PM,253599,10,Bio-PRF,,,12
4,Dec 21 2022 12:33PM,253604,10,"Nextsource Biotechnology, LLC",,,1
5,Dec 21 2022 12:25PM,253603,21,"NBTY Global, Inc.",,,1
6,Dec 21 2022 12:19PM,253602,10,"Goodwin Biotechnology, Inc.",,,1
7,Dec 21 2022 12:10PM,253601,10,"Snap Medical Industries, LLC",,,2
8,Dec 21 2022 12:03PM,253600,10,Bio-PRF,,,1
9,Dec 21 2022 12:01PM,253597,10,"Nextsource Biotechnology, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 1:14PM,253608,19,"Emersa Waterbox, LLC",3,,
1,Dec 21 2022 1:07PM,253607,22,"NBTY Global, Inc.",1,,
2,Dec 21 2022 12:53PM,253606,10,Yusen – 3D Matrix,2,,
3,Dec 21 2022 12:52PM,253599,10,Bio-PRF,12,,
4,Dec 21 2022 12:33PM,253604,10,"Nextsource Biotechnology, LLC",1,,
5,Dec 21 2022 12:25PM,253603,21,"NBTY Global, Inc.",1,,
6,Dec 21 2022 12:19PM,253602,10,"Goodwin Biotechnology, Inc.",1,,
7,Dec 21 2022 12:10PM,253601,10,"Snap Medical Industries, LLC",2,,
8,Dec 21 2022 12:03PM,253600,10,Bio-PRF,1,,
9,Dec 21 2022 12:01PM,253597,10,"Nextsource Biotechnology, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 1:14PM,253608,19,"Emersa Waterbox, LLC",3,,
1,Dec 21 2022 1:07PM,253607,22,"NBTY Global, Inc.",1,,
2,Dec 21 2022 12:53PM,253606,10,Yusen – 3D Matrix,2,,
3,Dec 21 2022 12:52PM,253599,10,Bio-PRF,12,,
4,Dec 21 2022 12:33PM,253604,10,"Nextsource Biotechnology, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 1:14PM,253608,19,"Emersa Waterbox, LLC",3.0,NaN,NaN
1,Dec 21 2022 1:07PM,253607,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Dec 21 2022 12:53PM,253606,10,Yusen – 3D Matrix,2.0,NaN,NaN
3,Dec 21 2022 12:52PM,253599,10,Bio-PRF,12.0,NaN,NaN
4,Dec 21 2022 12:33PM,253604,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 1:14PM,253608,19,"Emersa Waterbox, LLC",3.0,0.0,0.0
1,Dec 21 2022 1:07PM,253607,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Dec 21 2022 12:53PM,253606,10,Yusen – 3D Matrix,2.0,0.0,0.0
3,Dec 21 2022 12:52PM,253599,10,Bio-PRF,12.0,0.0,0.0
4,Dec 21 2022 12:33PM,253604,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5071768,86.0,5.0,0.0
12,253583,0.0,1.0,0.0
15,253592,1.0,0.0,0.0
16,253578,1.0,0.0,0.0
17,253591,1.0,0.0,0.0
19,1267890,84.0,10.0,1.0
20,507176,2.0,1.0,0.0
21,253603,1.0,0.0,0.0
22,253607,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5071768,86.0,5.0,0.0
1,12,253583,0.0,1.0,0.0
2,15,253592,1.0,0.0,0.0
3,16,253578,1.0,0.0,0.0
4,17,253591,1.0,0.0,0.0
5,19,1267890,84.0,10.0,1.0
6,20,507176,2.0,1.0,0.0
7,21,253603,1.0,0.0,0.0
8,22,253607,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,86.0,5.0,0.0
1,12,0.0,1.0,0.0
2,15,1.0,0.0,0.0
3,16,1.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,84.0,10.0,1.0
6,20,2.0,1.0,0.0
7,21,1.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,86.0
1,12,Released,0.0
2,15,Released,1.0
3,16,Released,1.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Executing,5.0,1.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0
Released,86.0,0.0,1.0,1.0,1.0,84.0,2.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,5.0,1.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0
2,Released,86.0,0.0,1.0,1.0,1.0,84.0,2.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,5.0,1.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0
2,Released,86.0,0.0,1.0,1.0,1.0,84.0,2.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()